# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [131]:
df = pd.read_csv("./data/AviationData.csv", encoding="latin1", low_memory=False)

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [132]:
# Cleaning tasks around engine info, cleaning make and model, collision detection, and a few small items

def normalize_engine_type(df):
    """
    Normalizes NaN, UNK, Unknown, and Unk values in the 'Engine.Type' column.
    NONE and Hybrid rocket will be dropped in filtering stage
    Potential future steps:
    - Group turbine engine types (Turbo Prop, Turbo Shaft, Turbo Fan, Turbo Jet, Geared Turbofan).
    - Group less common engine types (Electric, LR, NONE, Hybrid Rocket).
    """
    def normalize_engine(engine):
        if pd.isna(engine) or engine in ["UNK", "Unknown", "Unk"]:
            return "Unknown"
        else:
            return engine

    df["Engine.Type"] = df["Engine.Type"].apply(normalize_engine)
    return df

def create_engine_configuration(df):
    """Creates a new 'Engine.Configuration' column with grouped engine counts."""
    def group_engines(count):
        if count == 1:
            return "Single"
        elif count == 2:
            return "Twin"
        elif count >= 3:
            return "Multi"
        elif count == 0:
            return "None"
        else:
            return "Unknown"  # Handles NaN and other unexpected values

    df["Engine.Configuration"] = df["Number.of.Engines"].apply(group_engines)
    return df
    
def _create_aircraft_involved_column(df):
    """
    Creates an 'Aircraft Involved' column indicating the number of aircraft involved in each incident.
    Examining the data we found a large number of midair collisions that share Event IDs.
    """
    event_id_counts = df['Event.Id'].value_counts().to_dict()
    df['Aircraft Involved'] = df['Event.Id'].map(event_id_counts)
    return df

def _simplify_aircraft_category(category):
    if category == "Airplane":
        return "Airplane"
    elif category in ["Helicopter", "Gyrocraft", "Powered-Lift"]:
        return "Rotorcraft"
    elif pd.isna(category) or category in ["Unknown", "UNK"]:
        return "Other/Unknown"
    else:
        return np.nan # Mark as NaN for to drop the oddbal types like Rocket
    
def misc_cleaning(df):
    """Performs miscellaneous cleaning tasks on the DataFrame."""

    # Convert Event.Date to datetime
    df["Event.Date"] = pd.to_datetime(df["Event.Date"], errors="coerce")

    # Fill NaN values in Amateur.Built
    # Change NaN to "No" because examining the data shows the models for these are Boeing, Cessna, etc
    df.loc[:, "Amateur.Built"] = df["Amateur.Built"].fillna("No")

    # Fill NaN values in Aircraft.damage
    df.loc[:, "Aircraft.damage"] = df["Aircraft.damage"].fillna("Unknown")

    # Clean make and model columns.
    df.loc[:, "Make"] = df["Make"].str.strip().str.title()
    merge_names = {"Air Tractor Inc": "Air Tractor", "Airbus Industrie": "Airbus", "Aviat Aircraft Inc": "Aviat", 
               "Cirrus Design Corp": "Cirrus", "Cirrus Design Corp.": "Cirrus",
               "Dehavilland": "De Havilland", "Ercoupe (Eng & Research Corp.)": "Ercoupe",
                 "Grumman Acft Eng Cor-Schweizer": "Grumman", "Grumman American": "Grumman",
                "Grumman-Schweizer": "Grumman", "Robinson Helicopter": "Robinson",
                  "Robinson Helicopter Company": "Robinson", "Rockwell International": "Rockwell"}

    df['Make'] = df['Make'].replace(merge_names)    
    df.loc[:, "Model"] = df["Model"].str.strip()
    # Make a combo column for convenience
    df['MakeModel'] = df['Make'] + ' ' + df['Model']

    df = _create_aircraft_involved_column(df)

    df["Aircraft.Category"] = df["Aircraft.Category"].apply(_simplify_aircraft_category)

    return df

In [133]:
# Passenger count and injury/fatality processing

def _clean_injury_counts(df):
    """
    Sets null or negative injury counts to 0 and converts to integers.
    """
    injury_columns = [
        "Total.Fatal.Injuries",
        "Total.Serious.Injuries",
        "Total.Minor.Injuries",
        "Total.Uninjured"
    ]

    for col in injury_columns:
        df[col] = df[col].fillna(0).astype(int)
        df[col] = df[col].apply(lambda x: 0 if x < 0 else x)
    return df

def _create_injury_and_passenger_columns(df):
    """
    Creates "Bad.Injuries" (fatal + serious) and "Total.Passengers" columns.
    """
    df["Bad.Injuries"] = df["Total.Fatal.Injuries"] + df["Total.Serious.Injuries"]
    df["Total.Passengers"] = (
        df["Total.Fatal.Injuries"]
        + df["Total.Serious.Injuries"]
        + df["Total.Minor.Injuries"]
        + df["Total.Uninjured"]
    )
    return df

def _create_final_fatalities(df):
    """
    Creates a 'Final.Fatalities' column, taking the maximum of parsed and total fatal injuries.
    """
    def extract_fatalities(severity):
        if isinstance(severity, str) and severity.startswith("Fatal(") :
            try:
                return int(severity[6:-1])
            # There was some number here but we couldn't parse, so we'll default to 1
            # This will later be compared against the Total.Fatal.Injuries column
            except ValueError:
                return 1
        elif severity == "Fatal":
            return 1
        else:
            return 0

    df["Parsed.Fatalities"] = df["Injury.Severity"].apply(extract_fatalities)
    df["Final.Fatalities"] = df[["Parsed.Fatalities", "Total.Fatal.Injuries"]].max(axis=1)
    df.drop("Parsed.Fatalities", axis=1, inplace=True)
    return df

def process_injury_data(df):
    """Processes injury data by cleaning and creating new columns."""
    df = _clean_injury_counts(df)
    df = _create_injury_and_passenger_columns(df)
    df = _create_final_fatalities(df)
    return df

In [134]:
def _simplify_weather(weather):
    """
    Simplifies weather conditions into "Good", "Bad", and "Unknown".

    VMC (Visual Meteorological Conditions) -> "Good"
    IMC (Instrument Meteorological Conditions) -> "Bad"
    NaN or UNK/Unk -> "Unknown" (Weather condition not recorded or unknown)
    """
    if isinstance(weather, str): #Check that weather is a string.
        if weather == "VMC":
            return "Good"
        elif weather == "IMC":
            return "Bad"
        elif pd.isna(weather) or weather in ["UNK", "Unk"]:
            return "Unknown"
        else:
            return "Unknown" # Handles cases we haven't seen.
    else:
        return "Unknown" # Handles cases where weather is not a string.

# This function expects a single element, so it must be used within apply.
def apply_simplified_weather_to_column(df):
    df["Weather.Condition"] = df["Weather.Condition"].apply(_simplify_weather)
    return df

In [135]:
# We'll use a functional programming approach to keep ourselves organized

cleaning_functions = [
    process_injury_data,
    normalize_engine_type,
    create_engine_configuration,
    apply_simplified_weather_to_column,
    misc_cleaning,
]

def apply_cleaning_functions(df, cleaning_functions):
    """Applies a list of cleaning functions to the DataFrame."""
    for func in cleaning_functions:
        df = func(df)
    return df

df = apply_cleaning_functions(df, cleaning_functions)
# We should still have 88889 records after cleaning

In [136]:
# Filtering 

# We're only interested in data starting 1983
# Drops ~3000 records, remain: 85289
df = df[df["Event.Date"] >= pd.Timestamp("1983-01-01")]
# Drop amateur build rows that were not involved in collisions.
# Drops ~8000 records, remain: 77183
df = df[((df["Amateur.Built"] == "No") | (df["Aircraft Involved"] > 1))] # bitwise or to compare series

# Drop rotorcraft rows that were not involved in collisions, keeping "Other" records
# Drops ~4000 records, remain: 73245
df = df[
    (df["Aircraft.Category"] == "Airplane") |
    (df["Aircraft.Category"] == "Other/Unknown") |
    (df["Aircraft Involved"] > 1)
]

# Remove these categories since they are not close to what we're looking for
df = df[~df["Engine.Type"].isin(["NONE", "Hybrid Rocket"])]

# Drop makes with fewer than 50 examples.
make_counts = df["Make"].value_counts()
makes_to_keep = make_counts[make_counts >= 50].index
df = df[df["Make"].isin(makes_to_keep)]

# Drop rows where Model is NaN, per instructions
df = df.dropna(subset=["Model"])

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [137]:
# Done in previous cells

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [138]:
# Done in previous cells

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [139]:
# Done in previous cells

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [140]:
# Done in previous cells

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [141]:
# Done in previous cells

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [142]:
selected_columns = [
    "Event.Date", "Injury.Severity", "Aircraft.damage",
    "Make", "Model", "Amateur.Built",
    "Number.of.Engines", "Engine.Type", "Total.Fatal.Injuries",
    "Total.Serious.Injuries", "Total.Minor.Injuries", "Total.Uninjured",
    "Weather.Condition", "Broad.phase.of.flight", "Purpose.of.flight",
    "Aircraft.Category", "Aircraft Involved", "MakeModel", "Engine.Configuration", "Final.Fatalities"
]

df[selected_columns].to_csv("filtered_aircraft_data.csv", index=False)